In [2]:
#Flags to selectively run data imports for train and test. 0 = no import 1 = import
run_test=1
run_train=0

In [3]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

project = 'namshi-datahut' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

query1 = client.query(

'''


with cte_imp as (
  SELECT sku,
    imp_path,
      date,
      cast(imp_row as int) imp_row,
      cast(imp_col as int) imp_col,
      case when mod(cast(imp_row as int),2)=1 then (cast(imp_row as int)+1)/2 else (cast(imp_row as int))/2 end as imp_position,
      count(*) as total_impressions
FROM `namshi-datahut.sabio_silver.daily_product_imp_attr` 
WHERE  date >= current_date()-17 and date <= current_date()-3  and cast(imp_row as int) >0 and cast(imp_col as int) >0 and cast(imp_col as int) <=2 and imp_parent like "Categories%"
group by 1,2,3,4,5,6
order by 1,2 desc ,3 asc, 4 asc),

cte_click as (
  SELECT sku,
    imp_path,
      date,
      cast(imp_row as int) imp_row,
      cast(imp_col as int) imp_col,
      case when mod(cast(imp_row as int),2)=1 then (cast(imp_row as int)+1)/2 else (cast(imp_row as int))/2 end as imp_position,
      count(*) as total_clicks
FROM `namshi-datahut.sabio_silver.daily_product_imp_attr` 
WHERE  date >= current_date()-17 and date <= current_date()-3 and cast(imp_row as int) >0 and cast(imp_col as int) >0 and cast(imp_col as int) <=2 and imp_parent like "Categories%" and gv_timestamp is not null
group by 1,2,3,4,5,6
order by 1,2 desc ,3 asc, 4 asc),

cte_join as (
select sku,imp_path,imp_position,sum(total_impressions) as total_impressions, sum(total_clicks) as total_clicks
from 
(
select a.*,coalesce(b.total_clicks,0) as total_clicks
from cte_imp a
left join cte_click b
on a.sku = b.sku and a.date = b.date and a.imp_position = b.imp_position 
order by 2 desc, imp_position asc
)
group by 1,2,3)

select * from cte_join
where imp_position <=50
'''
)


query2 = client.query("""
    with cte_daily_prod_summary as 
(select distinct
a.date_key,
extract(dayofweek from a.date_key) as dayofweek,
extract(day from a.date_key) as dayofmonth,
case when extract(dayofweek from a.date_key) in (7,1) then 1 else 0 end weekend_flag,
a.sku_config,
a.ttl_clicks_AE,
a.ttl_impressions_AE,
coalesce(a.current_price,b.retail_price) as current_price,
round(daily_markup_discount_pct *100,0) discount_pct,
round(availability_dxb_pct,0) as availability_dxb_pct,
b.attribute_set,
b.brand_clean,
b.status,
b.retail_price,
b.activated_at,
date_diff(a.date_key,cast(b.activated_at as date),day) as time_since_launch,
relaunched_at,
date_diff(a.date_key,relaunched_at,day) as time_since_relaunch,
b.season,
b.season_grouped_ty,
b.season_number,
b.color,
b.color_family,
b.merch_gender,
b.department,
b.category_clean,
b.subcategory_clean,
b.gender_clean,
b.age_group,
b.inventory_group_clean,
b.occasion,
b.basic_type,
b.specialist,
b.product_detail,
b.merch_team_clean,
b.world_tag,
b.upper_material,
b.lining_material,
b.sole_material,
a.ageing
from `namshi-datagrill.sabio_gold.daily_product_summary` a
left join `namshi-datahut.sabio_silver.item_d` b
 on a.sku_config = b.sku_config
 where  a.date_key >= current_date()-17 and a.date_key <= current_date()-3 
 order by a.date_key desc,a.ttl_clicks_AE desc--order by date_key desc   
)

select 

sku_config,
attribute_set,
brand_clean,
status,
inventory_group_clean,
occasion,
basic_type,
specialist,
product_detail,
merch_team_clean,
world_tag,
upper_material,
lining_material,
sole_material,
season,
season_grouped_ty,
season_number,
color,
color_family,
merch_gender,
department,
category_clean,
subcategory_clean,
gender_clean,
max(cast(activated_at as date)) activated_at,
max(cast(relaunched_at as date)) relaunched_at,
date_diff(current_date(),max(cast(activated_at as date)),day) as time_since_launch,
max(ageing) as time_since_relaunch,
avg(current_price) current_price,
sum(ttl_clicks_AE) as ttl_clicks_AE ,
sum(ttl_impressions_AE) as ttl_impressions_AE,
round(safe_divide(sum(ttl_clicks_AE),sum(ttl_impressions_AE))*100,2) as click_through_rate,
round(avg(discount_pct),2) discount_pct,
round(avg(availability_dxb_pct),2) as availability_dxb_pct,
from cte_daily_prod_summary
group by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24

  """)

query3 = client.query("""
    with cte_imp as (
  SELECT sku,
    imp_path,
      date,
      cast(imp_row as int) imp_row,
      cast(imp_col as int) imp_col,
      case when mod(cast(imp_row as int),2)=1 then (cast(imp_row as int)+1)/2 else (cast(imp_row as int))/2 end as imp_position,
      count(*) as total_impressions
FROM `namshi-datahut.sabio_silver.daily_product_imp_attr` 
WHERE  date >= current_date()-32 and date <= current_date()-18  and cast(imp_row as int) >0 and cast(imp_col as int) >0 and cast(imp_col as int) <=2 and imp_parent like "Categories%"
group by 1,2,3,4,5,6
order by 1,2 desc ,3 asc, 4 asc),

cte_click as (
  SELECT sku,
    imp_path,
      date,
      cast(imp_row as int) imp_row,
      cast(imp_col as int) imp_col,
      case when mod(cast(imp_row as int),2)=1 then (cast(imp_row as int)+1)/2 else (cast(imp_row as int))/2 end as imp_position,
      count(*) as total_clicks
FROM `namshi-datahut.sabio_silver.daily_product_imp_attr` 
WHERE  date >= current_date()-32 and date <= current_date()-18 and cast(imp_row as int) >0 and cast(imp_col as int) >0 and cast(imp_col as int) <=2 and imp_parent like "Categories%" and gv_timestamp is not null
group by 1,2,3,4,5,6
order by 1,2 desc ,3 asc, 4 asc),

cte_join as (
select sku,imp_path,imp_position,sum(total_impressions) as total_impressions, sum(total_clicks) as total_clicks
from 
(
select a.*,coalesce(b.total_clicks,0) as total_clicks
from cte_imp a
left join cte_click b
on a.sku = b.sku and a.date = b.date and a.imp_position = b.imp_position 
order by 2 desc, imp_position asc
)
group by 1,2,3)

select * from cte_join

where imp_position <=50 and (imp_path like '%shoe%')
  """)


test_query1 =  client.query( 
    """
    with cte_imp as (
  SELECT sku,
    imp_path,
      date,
      cast(imp_row as int) imp_row,
      cast(imp_col as int) imp_col,
      case when mod(cast(imp_row as int),2)=1 then (cast(imp_row as int)+1)/2 else (cast(imp_row as int))/2 end as imp_position,
      count(*) as total_impressions
FROM `namshi-datahut.sabio_silver.daily_product_imp_attr` 
WHERE  date >= current_date()-2 and date <= current_date()-1  and cast(imp_row as int) >0 and cast(imp_col as int) >0 and cast(imp_col as int) <=2 and imp_parent like "Categories%"
group by 1,2,3,4,5,6
order by 1,2 desc ,3 asc, 4 asc),

cte_click as (
  SELECT sku,
    imp_path,
      date,
      cast(imp_row as int) imp_row,
      cast(imp_col as int) imp_col,
      case when mod(cast(imp_row as int),2)=1 then (cast(imp_row as int)+1)/2 else (cast(imp_row as int))/2 end as imp_position,
      count(*) as total_clicks
FROM `namshi-datahut.sabio_silver.daily_product_imp_attr` 
WHERE  date >= current_date()-2 and date <= current_date()-1 and cast(imp_row as int) >0 and cast(imp_col as int) >0 and cast(imp_col as int) <=2 and imp_parent like "Categories%" and gv_timestamp is not null
group by 1,2,3,4,5,6
order by 1,2 desc ,3 asc, 4 asc),

cte_join as (
select sku,imp_path,imp_position,sum(total_impressions) as total_impressions, sum(total_clicks) as total_clicks
from 
(
select a.*,coalesce(b.total_clicks,0) as total_clicks
from cte_imp a
left join cte_click b
on a.sku = b.sku and a.date = b.date and a.imp_position = b.imp_position 
order by 2 desc, imp_position asc
)
group by 1,2,3)

select * from cte_join
where imp_position <=50
  """
  )

test_query2 =  client.query(
    """
    with cte_daily_prod_summary as 
(select distinct
a.date_key,
extract(dayofweek from a.date_key) as dayofweek,
extract(day from a.date_key) as dayofmonth,
case when extract(dayofweek from a.date_key) in (7,1) then 1 else 0 end weekend_flag,
a.sku_config,
a.ttl_clicks_AE,
a.ttl_impressions_AE,
coalesce(a.current_price,b.retail_price) as current_price,
round(daily_markup_discount_pct *100,0) discount_pct,
round(availability_dxb_pct,0) as availability_dxb_pct,
b.attribute_set,
b.brand_clean,
b.status,
b.retail_price,
b.activated_at,
date_diff(a.date_key,cast(b.activated_at as date),day) as time_since_launch,
relaunched_at,
date_diff(a.date_key,relaunched_at,day) as time_since_relaunch,
b.season,
b.season_grouped_ty,
b.season_number,
b.color,
b.color_family,
b.merch_gender,
b.department,
b.category_clean,
b.subcategory_clean,
b.gender_clean,
b.age_group,
b.inventory_group_clean,
b.occasion,
b.basic_type,
b.specialist,
b.product_detail,
b.merch_team_clean,
b.world_tag,
b.upper_material,
b.lining_material,
b.sole_material,
a.ageing
from `namshi-datagrill.sabio_gold.daily_product_summary` a
left join `namshi-datahut.sabio_silver.item_d` b
 on a.sku_config = b.sku_config
 where  a.date_key >= current_date()-2 and a.date_key <= current_date()-1 
 order by a.date_key desc,a.ttl_clicks_AE desc--order by date_key desc   
)

select 

sku_config,
attribute_set,
brand_clean,
status,
inventory_group_clean,
occasion,
basic_type,
specialist,
product_detail,
merch_team_clean,
world_tag,
upper_material,
lining_material,
sole_material,
season,
season_grouped_ty,
season_number,
color,
color_family,
merch_gender,
department,
category_clean,
subcategory_clean,
gender_clean,
max(cast(activated_at as date)) activated_at,
max(cast(relaunched_at as date)) relaunched_at,
date_diff(current_date(),max(cast(activated_at as date)),day) as time_since_launch,
max(ageing) as time_since_relaunch,
avg(current_price) current_price,
sum(ttl_clicks_AE) as ttl_clicks_AE ,
sum(ttl_impressions_AE) as ttl_impressions_AE,
round(safe_divide(sum(ttl_clicks_AE),sum(ttl_impressions_AE))*100,2) as click_through_rate,
round(avg(discount_pct),2) discount_pct,
round(avg(availability_dxb_pct),2) as availability_dxb_pct,
from cte_daily_prod_summary
group by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24

  """
  )

test_query3 = client.query( 
    """
    with cte_imp as (
  SELECT sku,
    imp_path,
      date,
      cast(imp_row as int) imp_row,
      cast(imp_col as int) imp_col,
      case when mod(cast(imp_row as int),2)=1 then (cast(imp_row as int)+1)/2 else (cast(imp_row as int))/2 end as imp_position,
      count(*) as total_impressions
FROM `namshi-datahut.sabio_silver.daily_product_imp_attr` 
WHERE  date >= current_date()-10 and date <= current_date()-3  and cast(imp_row as int) >0 and cast(imp_col as int) >0 and cast(imp_col as int) <=2 and imp_parent like "Categories%"
group by 1,2,3,4,5,6
order by 1,2 desc ,3 asc, 4 asc),

cte_click as (
  SELECT sku,
    imp_path,
      date,
      cast(imp_row as int) imp_row,
      cast(imp_col as int) imp_col,
      case when mod(cast(imp_row as int),2)=1 then (cast(imp_row as int)+1)/2 else (cast(imp_row as int))/2 end as imp_position,
      count(*) as total_clicks
FROM `namshi-datahut.sabio_silver.daily_product_imp_attr` 
WHERE  date >= current_date()-10 and date <= current_date()-3 and cast(imp_row as int) >0 and cast(imp_col as int) >0 and cast(imp_col as int) <=2 and imp_parent like "Categories%" and gv_timestamp is not null
group by 1,2,3,4,5,6
order by 1,2 desc ,3 asc, 4 asc),

cte_join as (
select sku,imp_path,imp_position,sum(total_impressions) as total_impressions, sum(total_clicks) as total_clicks
from 
(
select a.*,coalesce(b.total_clicks,0) as total_clicks
from cte_imp a
left join cte_click b
on a.sku = b.sku and a.date = b.date and a.imp_position = b.imp_position 
order by 2 desc, imp_position asc
)
group by 1,2,3)

select * from cte_join
where imp_position <=50 and (imp_path like '%clothing%' or imp_path like '%shoe%' or imp_path like '%tops%')
  """)

In [6]:
client = bigquery.Client()
if run_train==1:
  df_query1 = query1.result().to_dataframe()
  print("Total unique queries for train query 1: ", df_query1['imp_path'].nunique())
  print("Shape of the data  for train query 1: ",df_query1.shape )
  df_query2 = query2.result().to_dataframe()
  print("Shape of the data  for train query 2: ",df_query2.shape )
  df_query3 = query3.result().to_dataframe()
  print("Total unique queries train for query 3: ", df_query3['imp_path'].nunique())
  print("Shape of the data  for train query 3: ",df_query3.shape )
  df_query1.to_pickle('drive/MyDrive/Training Data/df_query1.pkl')
  df_query2.to_pickle('drive/MyDrive/Training Data/df_query2.pkl')
  df_query3.to_pickle('drive/MyDrive/Training Data/df_query3.pkl')

print('\n')

if run_test==1:
  df_query1_test = test_query1.result().to_dataframe()
  print("Total unique queries for test query 1: ", df_query1_test['imp_path'].nunique())
  print("Shape of the data  for test query 1: ",df_query1_test.shape )
  df_query2_test = test_query2.result().to_dataframe()
  print("Shape of the data  for test query 2: ",df_query2_test.shape )
  df_query3_test = test_query3.result().to_dataframe()
  print("Total unique queries for test query 3: ", df_query3_test['imp_path'].nunique())
  print("Shape of the data  for test query 3: ",df_query3_test.shape )
  
  df_query1_test.to_pickle('drive/MyDrive/Training Data/df_query1_test.pkl')
  df_query2_test.to_pickle('drive/MyDrive/Training Data/df_query2_test.pkl')
  df_query3_test.to_pickle('drive/MyDrive/Training Data/df_query3_test.pkl')


